# Python is for science!

There are many cool libraries available to you in Python. These include `numpy` and `matplotlib` (used in this notebook) and scipy. `numpy`, which accesses very fast compiled code, is particularly good for numerical computing, `matplotlib` allows the quick production of visually appealing and useful plots (many of the plots you will see nowadays are produced with matplotlib), and `scipy` contains an array of scientific functionality. All of these are installed on the Docker container you downloaded and you will see them in use in the workshops this week.

In this notebook, we aren't hoping to preempt the other sessions, but will proceed through some useful tasks you might wish to do from time to time, demonstrating reading from `PSRFITS` files, inspecting the header, putting the binary data into a numpy array, plotting a dynamic spectrum and then manipulating the plot to enhance clarity of the displayed output. The file in question is in the same directory as this notebook; it's a small file (already subbanded) of a bright pulsar, taken at Arecibo Observatory with `puppi`, in search mode.

Examples here are from Michael Lam.

## Import statements

Here we import the useful stuff. `numpy` is nearly always aliased as `np` and `astropy.io.fits` is often aliased as `pyfits` because `pyfits` is an older package, used in a lot of legacy code, that was rolled into `astropy`'s `astropy.io.fits` module (using this alias means that a lot of that old code still works!). The `cm` module in `matplotlib` is for colormaps, and the `pyplot` provides us with a plotting functionality similar to that in `MATLAB`. Lastly, the `%matplotlib inline` directive, which is specific to `Jupyter`, lets us plot inside the notebook cells rather than popping up a fresh window for each plot.

It's really very impressive, how much cool functionality is given to use by just importing those few modules.

In [ ]:
%matplotlib inline
import numpy as np
import astropy.io.fits as pyfits
import matplotlib.pyplot as plt
import matplotlib.cm as cm

## Do you have the file?

We use a Jupyter directive to get a list of the files in the current working directory:

In [ ]:
!ls

If `puppi_57166_J1404+12_1326_subs_0001.fits` isn't in there, you need to get it:

In [ ]:
!wget http://ipta.phys.wvu.edu/files/student-week-2018/puppi_57166_J1404+12_1326_subs_0001.fits

## Load the fits file via astropy

Sometimes PSRFITS files are missing the END card so we set `ignore_missing_end = True` just in case. This is something which can save you frustrating errors, particularly when running a pipeline on very many files. The file is in the same directory as this notebook; as mentioned above, it's a subbanded search-mode file of a bright pulsar; the period of this pulsar, `J1404+12`, is around ~2.65 seconds and the DM is 18.836.

You can read the `astropy.io.fits` documentation, with examples, [at this page](http://docs.astropy.org/en/stable/io/fits/) (where you also see how to use it if you aren't aliasing to `pyfits`)

In [ ]:
#Should include full filepath if it's not in the local directory
filename = "puppi_57166_J1404+12_1326_subs_0001.fits"
#the open function returns a list of the Header/Data Units (HDUs) in the file
hdulist = pyfits.open(filename,ignore_missing_end=True)

## Look at the data shape

Sometimes you don't know what to expect, so this is how you can see what's in the FITS file. The first HDU is the `PRIMARY` HDU. The basic FITS standard is explained briefly [here](https://fits.gsfc.nasa.gov/fits_primer.html), but PSRFITS is an extension of this (and in some cases, the data values are encoded with less than 8 bits and packed into 8 bit `ints`, which means they need to be unpacked; this file, however, contains 8-bit values). Here we learn that there are two HDUs and get information about their contents.

The first thing you'll want to try to understand the structure of a FITS file is to use the `info()` function, like so:

In [ ]:
# Show useful information about the tables in the FITS file
hdulist.info()

You can see the names of the different tables; in this case there are two, `PRIMARY` and `SUBINT`, which you can either call by name like `hdulist['PRIMARY']` or number like `hdulist[0]`. There is always a primary header and then there can be multiple tables stores.

For any of the tables, you can use `TABLE.header` to grab header information. The header for `PRIMARY` is "the" header for the FITS file but tables can store their own headers. Let's look at the primary header:

In [ ]:
# Show the header in the primary HDU: typically the observation parameters are in here
hdulist['PRIMARY'].header

### Where is my data?

The data are in the `SUBINT` HDU. Let's first look at the header. You can see that there's a lot of different fields of information, what the shapes/formats of the data are, etc.

In [ ]:
hdulist['SUBINT'].header

For any table with data, you can use `TABLE.data` to grab the data. Let's look at the data

In [ ]:
# Grab the data
data = hdulist['SUBINT'].data
# Look at the first subintegration
print(data[0])

You can try to match what's in the header to what's in the data table. You'll notice that the last element of this first subintegration is the full data array of pulse intensities. Before that are a bunch of other items that you can see in the header, such as the length of the subintegration (in units of seconds, note TUNIT1), the RA/dec position of the telescope, etc. Let's explicitly grab the length of the subintegration using the flag indicated in the header.

In [ ]:
hdulist['SUBINT'].data['TSUBINT']

So, each of the 68 subintegrations has 2.684 seconds worth of data. 

Now let's look at the shape of the intensity data. These shapes can be derived from the header as well but sometimes you'll just want to grab these in code rather than just printing them out

In [ ]:
# number of subintegrations
print(np.shape(data))
# shape of each subintegration (the last value is the actual array of intensities)
# shape is in (time, polarization, frequency, phase)
print(np.shape(data[0][-1]))
# Remove the dimensions with only one value, which makes a dynamic spectrum in (time, frequency)
print(np.shape(data[0][-1].squeeze()))

So, there are 68 subintegrations, each with about 2.684 seconds of data. In each of these subintegrations, there are 32768 time samples, 1 polarization, 128 frequency channels, and 1 phase bin. Note that since this is a search-mode observation rather than a fold-mode observation, there's no pulse period and so no phase!

## Plot the dynamic spectrum of the first subintegration

The dynamic spectrum shows the intensities as a function of time and frequency. Given the shape of the squeezed array above, that's very easy to do in matplotlib with the `imshow()` function. For convenience, we've added some useful arguments to the function. You can also pick your own [colormap](https://matplotlib.org/2.0.2/examples/color/colormaps_reference.html)!

Side programming note: because you should think of a 2D array with indices as (row, column), which is like (y,x), to plot in (x,y), you'll often want to transpose the array as we have done below.

In [ ]:
dynspec = data[0][-1].squeeze()
# Transpose the array to go from (row, column) -> (x,y)
plt.imshow(np.transpose(dynspec),origin='lower',interpolation='nearest',aspect='auto',cmap=cm.inferno)
plt.xlabel('Time Bin')
plt.ylabel('Frequency Channel')

## Reduce the array in time

The following is a simple function to average a 2D array in the first dimension (time in this case) by a certain factor. Then we will plot the reduced dynamic spectrum like above.

In [ ]:
def timeaverager(array,window_len):
    if window_len==1:
        return array
    T,F = np.shape(array)
    retval = np.zeros((T//window_len,F)) #integer division in python 3!
    counts = np.zeros_like(retval)
    for i in range(window_len):
        retval += array[i:T:window_len,:]
    return retval/window_len

In [ ]:
# Average down by a factor of 8 in time
avgdynspec = timeaverager(dynspec,8)
# Transpose the array to go from (row, column) -> (x,y)
plt.imshow(np.transpose(avgdynspec),origin='lower',interpolation='nearest',aspect='auto',cmap=cm.inferno)
plt.xlabel('Time Bin')
plt.ylabel('Frequency Channel')